In [38]:
from pyspark.mllib.fpm import FPGrowth
from operator import add
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import date_format
from pyspark.sql.types import *
import pandas as pd
from pyspark.sql.functions import unix_timestamp, from_unixtime
from pyspark.sql.functions import concat_ws
#import plotly.graph_objs as go
#from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import shutil
#init_notebook_mode(connected=True)
spark = SparkSession.builder.appName("Python Spark SQL basic example").getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)
df = sqlContext.read.load('/test_dev/mba-code/dataset.csv', format='csv', header='true', inferSchema='true',encoding='UTF-8')
df.select("InvoiceNo", "StockCode", "Description", "Quantity", "InvoiceDate","InvoiceDateWS" ,date_format(from_unixtime(unix_timestamp('InvoiceDateWS', 'mm/dd/yyy')), 'EEEE').alias('weekday'),"CustomerID","Country").write.save("Invoices.parquet", format="parquet")
parquetFile = spark.read.parquet("Invoices.parquet")
# Parquet files can also be used to create a temporary view and then used in SQL statements.
parquetFile.createOrReplaceTempView("parquetFile")
DescriptionGrp = spark.sql("SELECT distinct InvoiceNo,StockCode FROM parquetFile group by InvoiceNo,StockCode")
#print(DescriptionGrp.rdd.take(2))
minSupport = 0.05 * DescriptionGrp.rdd.count()
apr_tem=DescriptionGrp.rdd.map(lambda x: (x[0], list([x[1]]))).reduceByKey(lambda x,y: x + y)
transactions_fp=apr_tem.map(lambda x: (x[1]))
print(transactions_fp.take(2))
#schema = StructType([StructField("test_123",ArrayType(StringType(),True),True)])
#fields = [StructField(field_name, StringType(), True) for field_name in schema.split(',')]
#schema = StructType(fields)
final_transactions_rdd = sc.parallelize(transactions_fp.collect())
final_transactions = [",".join(map(str, item)) for item in final_transactions_rdd.collect()]
transactions = spark.createDataFrame([final_transactions])
#transactions_test = final_transactions.map(lambda line: line.strip().split(','))
#print(transactions_test)
model = FPGrowth.train(transactions, minSupport=minSupport, numPartitions=10)
result = model.freqItemsets()
print result.collect()
#for fi in result.collect():
#    print(fi)

[[u'22667', u'22865', u'22666', u'22114', u'23212', u'22904', u'22348', u'20914'], [u'22665', u'21733', u'21078', u'48194', u'21041', u'22904', u'21080', u'21654', u'22469', u'23321', u'21899', u'23178', u'22666', u'22193', u'22427', u'22507', u'20914', u'22095', u'23302', u'84596G', u'22667', u'21164', u'20832', u'21294', u'84974', u'85123A', u'84596B', u'23198', u'23106', u'21218', u'22097', u'20676', u'22094', u'21034']]


Py4JError: An error occurred while calling o3125.trainFPGrowthModel. Trace:
py4j.Py4JException: Method trainFPGrowthModel([class org.apache.spark.sql.Dataset, class java.lang.Double, class java.lang.Integer]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:318)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:326)
	at py4j.Gateway.invoke(Gateway.java:274)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:238)
	at java.lang.Thread.run(Thread.java:748)



The analysis is interesting, and useful for market basket analysis. Imagine the letters are skus instead!